## Test rf

In [2]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_RF(unittest.TestCase):
    
    def test_rf_inputtrain(self):
        train = pd.read_csv('../Arranged_Data/final_weater.csv')[[
            'State', 'TotalMonthlyPrecip', 'TempSummer',
            'TempWinter', 'Avgwindspeed']]
        #test if DC exists
        self.assertIn('DC',list(train.State))
        #test if there is nan in input data
        with self.assertRaises(InputNan):
            train.isnull().sum().sum() != 0
        
    def test_rf_inputpara(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        #test the type of input data
        self.assertIsInstance(prec,int or np.float64)
        self.assertIsInstance(ts,int or np.float64)
        self.assertIsInstance(tw,int or np.float64)
        self.assertIsInstance(ws,int or np.float64)
        #test the limitation of input dataset
        with self.assertRaises(TemperatureInputOutOfLimitation):
            ts > 110 or ts < tw or tw < -10
        with self.assertRaises(TemperatureInputOutOfLimitation):
            prec < 0 
        with self.assertRaises(TemperatureInputOutOfLimitation):
            ws < 0 

    def test_rf_output(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        output = ease.rf(prec, ts, tw, ws)
        #test the type of output dataset
        self.assertIsInstance(output,dict)
        #test the output values
        with self.assertRaises(ValueError):
            for i in list(output.values()):
                0 < i < 1
                
#if __name__ == '__main__':
    #unittest.main()

## Test avg_capacity

In [1]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_avg_capacity(unittest.TestCase):
    
    def test_avg_cap_input(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        #test if there is nan in input dataframe
        with self.assertRaises(InputNan):
            average_plant_capacity.isnull().sum().sum() != 0
    
    def test_avg_cap_output(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        vote = ease.rf(prec, ts, tw, ws)
        output = avg_capacity(vote)
        #test the type of output dataset
        self.assertIsInstance(output,list)
        #test the length of output dataset
        self.assertEqual(len(output),6) 
#if __name__ == '__main__':
    #unittest.main()

In [11]:
vote = ease.rf(3, 69, 35, 5.4)
ease.avg_capacity(vote)

[2551245.8119999995,
 286664.9808,
 62748.209599999995,
 64409.339799999994,
 70.3428,
 129394.76900000001]

## Test possible_type

In [3]:
avg_cap_list = [2505375.75, 287825.37, 62167.29, 56935.33, 89.96000000000001, 126547.58]
a = ease.possible_type(avg_cap_list)

In [7]:
ease.possible_type(avg_cap_list)

[[0.99999999980861232, 2505375.75, 'Coal'],
 [-0.94041387467636262, 287825.37, 'NG'],
 [0.9975138031215941, 62167.29, 'Petro'],
 [0.99963990872898434, 126547.58, 'Wind']]

In [4]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_possible_type(unittest.TestCase):

    def test_possible_type_input(self):
        cap_pop = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        #test if there is nan in input capacity dataframe
        with self.assertRaises(InputNan):
            cap_pop.isnull().sum().sum() != 0
        
    def test_possible_type_output(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        vote = ease.rf(prec, ts, tw, ws)
        avg_cap_list = ease.avg_capacity(vote)
        output = ease.possible_type(avg_cap_list)
        
        #test the type of output dataset
        self.assertIsInstance(output,list)
        
        #calculate how many clean energy resource have been filtered
        type_list = []
        for i in range(len(ease.possible_type(avg_cap_list))):
            type = ease.possible_type(avg_cap_list)[i][-1]
            type_list.append(type)
            clean_ = ['Hydro', 'Solar', 'Wind']
        filtered = []
        for i in clean_:
            if i in type_list:
                pass
            else:
                filtered.append(i)
        #test if all the clean resources have been filtered out
        with self.assertRaises(AllCleanEnergyFilteredOut):
            len(filtered) == 3

#if __name__ == '__main__':
    #unittest.main()

## Test rf_fluctuation

In [7]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_rf_fluctuation(unittest.TestCase):
    
    def test_rf_fluctuation_output(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        output = rf_fluctuation(prec, ts, tw, ws)
        
        #test the type of output
        self.assertIsInstance(output,list)
        #test the length of output
        self.assertEqual(len(output),3) 
        #test fluctuation level
        std = output[1]
        with self.assertRaises(OverFluctuation):
            std > 0.01 # not sure whether std = 0.01 is a proper level

if __name__ == '__main__':
    unittest.main()